**使用 SavedModel 完整匯出模型**

Keras 模型可以方便的匯出為 SavedModel 格式。不過需要註意的是，因為 SavedModel 基於計算圖，所以對於使用繼承 tf.keras.Model 類建立的 Keras 模型，其需要匯出到 SavedModel 格式的方法（比如 call ）都需要使用 @tf.function 修飾（ @tf.function 的使用方式見 前文 ）。然後，假設我們有一個名為 model 的 Keras 模型，使用下面的程式碼即可將模型匯出為 SavedModel：

tf.saved_model.save(model, "保存的目標資料夾名稱")

在需要載入 SavedModel 文件時，使用

model = tf.saved_model.load("保存的目標資料夾名稱")

以下是一個簡單的範例，將 前文 MNIST 手寫體識別的模型 進行導出和導入。
導出模型到 saved/1 文件夾：

In [ ]:
!mkdir save1

In [ ]:
import tensorflow as tf
import numpy as np
#from zh.model.utils import MNISTLoader

class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的圖片預設為uint8（0-255的數字）。以下程式碼將其正規化到0-1之間的浮點數，並在最後增加一維作為顏色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 從資料集中隨機取出batch_size個元素並返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

num_epochs = 1
batch_size = 50
learning_rate = 0.001

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax()
])

data_loader = MNISTLoader()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)
model.fit(data_loader.train_data, data_loader.train_label, epochs=num_epochs, batch_size=batch_size)
tf.saved_model.save(model, "saved/1")

1200/1200 [==============================] - 2s 2ms/step - loss: 0.3027 - sparse_categorical_accuracy: 0.9153
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved/1/assets


將 saved/1 中的模型匯入並測試性能：

In [ ]:
import tensorflow as tf
#from zh.model.utils import MNISTLoader

batch_size = 50

model = tf.saved_model.load("saved/1")
data_loader = MNISTLoader()
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)

for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.945700


使用繼承 tf.keras.Model 類別建立的 Keras 模型同樣可以以相同方法匯出，只需註意 call 方法需要以 @tf.function 修飾，以轉化為 SavedModel 支持的計算圖，程式碼如下：

In [ ]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    @tf.function
    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

num_epochs = 5
batch_size = 50
learning_rate = 0.001

model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

num_batches = int(data_loader.num_train_data // batch_size * num_epochs)

for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

tf.saved_model.save(model, "saved/2")

串流輸出內容已截斷至最後 5000 行。
batch 1002: loss 0.307825
batch 1003: loss 0.449200
batch 1004: loss 0.235517
batch 1005: loss 0.299323
batch 1006: loss 0.186098
batch 1007: loss 0.268106
batch 1008: loss 0.304821
batch 1009: loss 0.249487
batch 1010: loss 0.163856
batch 1011: loss 0.138494
batch 1012: loss 0.333458
batch 1013: loss 0.218079
batch 1014: loss 0.292033
batch 1015: loss 0.054405
batch 1016: loss 0.196721
batch 1017: loss 0.149121
batch 1018: loss 0.058366
batch 1019: loss 0.081509
batch 1020: loss 0.215794
batch 1021: loss 0.148949
batch 1022: loss 0.153338
batch 1023: loss 0.161995
batch 1024: loss 0.278869
batch 1025: loss 0.384291
batch 1026: loss 0.246948
batch 1027: loss 0.090874
batch 1028: loss 0.401067
batch 1029: loss 0.248946
batch 1030: loss 0.194989
batch 1031: loss 0.128825
batch 1032: loss 0.108417
batch 1033: loss 0.158208
batch 1034: loss 0.097861
batch 1035: loss 0.253557
batch 1036: loss 0.072316
batch 1037: loss 0.161240
batch 1038: loss 0.261790
batch 1039: loss 

對於使用繼承 tf.keras.Model 類建立的 Keras 模型 model ，使用 SavedModel 載入後將無法使用 model() 直接進行推斷，而需要使用 model.call() 。

In [ ]:
model = tf.saved_model.load("saved/2")

sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)

for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.call(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.971600


**Keras Sequential save 方法（Jinpeng）**

執行過程會比較久，執行結束後，會在當前目錄產生 mnist_cnn.h5 文件（HDF5 格式），就是 keras 訓練後的模型，其中已經包含了訓練後的模型結構和權重值等資訊。

在伺服器端，可以直接透過 keras.models.load_model("mnist_cnn.h5") 載入，然後進行推論；在移動設備端需要將 HDF5 模型文件轉換為 TensorFlow Lite 的格式，然後透過相應平台的 Interpreter 載入，然後進行推論。

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax()
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

model.fit(data_loader.train_data, data_loader.train_label, epochs=num_epochs, batch_size=batch_size)

model.save('keras_seq.h5')

Epoch 1/5
1200/1200 [==============================] - 2s 2ms/step - loss: 0.3016 - sparse_categorical_accuracy: 0.9153
Epoch 2/5
1200/1200 [==============================] - 2s 2ms/step - loss: 0.1396 - sparse_categorical_accuracy: 0.9590
Epoch 3/5
1200/1200 [==============================] - 2s 2ms/step - loss: 0.0971 - sparse_categorical_accuracy: 0.9717
Epoch 4/5
1200/1200 [==============================] - 2s 2ms/step - loss: 0.0742 - sparse_categorical_accuracy: 0.9783
Epoch 5/5
1200/1200 [==============================] - 2s 2ms/step - loss: 0.0588 - sparse_categorical_accuracy: 0.9828


In [ ]:
model = tf.keras.models.load_model('keras_seq.h5')

sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)

for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.974600
